In [12]:
import numpy as np
from numpy import *
import scipy
from scipy.special import erf

In [13]:
def xyz_reader(file_name):
    file = open(file_name, 'r')

    number_of_atoms = 0
    atom_type = []
    atom_coordinates = []

    for idx, line in enumerate(file):
        if idx == 0:
            try:
                number_of_atoms = line.split()[0]
            except:
                print("file not in correct format.")
        
        if idx == 1:
            continue

        if idx != 0:
            split = line.split()
            atom = split[0]
            coordinates = [float(split[1]),
                           float(split[2]),
                           float(split[3])]
            atom_type.append(atom)
            atom_coordinates.append(coordinates)

    file.close()

    return number_of_atoms, atom_type, atom_coordinates

In [14]:
file_name = "../data/xyz/HeH.xyz"
number_of_atoms, atoms, atom_coordinates = xyz_reader(file_name)

In [15]:
atoms

['He', 'H']

In [16]:
atom_coordinates

[[0.0, 0.0, 0.0], [0.0, 0.0, 1.4632]]

In [17]:
STOnG = 3

zeta_dict = {'H': [1.24],
             'He': [2.0925],
             'Li': [2.69, 0.80],
             'Be': [3.68, 1.15],
             'B': [4.68, 1.50],
             'C': [5.67, 1.72]}

max_quantum_number = {'H': 1,
                      'He': 1,
                      'Li': 2,
                      'Be': 2,
                      'C': 2}

D = np.array([[0.444635, 0.535328, 0.154329],
              [0.700115, 0.399513, -0.0999672]])

alpha = np.array([[0.109818, 0.405771, 2.22766],
                  [0.0751386, 0.231031, 0.994203]])

B = 0
for atom in atoms:
    B += max_quantum_number[atom]


In [18]:
N = 2

charge_dict = {'H': 1,
               'He': 2,
               'Li': 3,
               'Be': 4,
               'B': 5,
               'C': 6,
               'N': 7,
               'O': 8,
               'F': 9,
               'Ne': 10}

In [1]:
def gauss_product(gauss_A, gauss_B):
    a, Ra = gauss_A
    b, Rb = gauss_B
    p = a + b
    diff = np.linalg.norm(Ra - Rb)**2
    N = (4 * a * b / (np.pi**2))**0.75
    K = N * exp(-a * b / p * diff)
    Rp = (a * Ra + b * Rb) / p
    
    return p, diff, K, Rp

In [3]:
def overlap(A, B):
    p, diff, K, Rp = gauss_product(A, B)
    prefactor = (np.pi / p)**1.5
    return prefactor * K

def kinetic(A, B):
    p, diff, K, Rp = gauss_product(A, B)
    prefactor = (np.pi / p)**1.5
    
    a, Ra = A
    b, Rb = B
    reduced_exponent = a * b / p
    return reduced_exponent * (3 - 2 * reduced_exponent * diff) * prefactor * K

In [4]:
def Fo(t):
    if t == 0:
        return 1
    else:
        return (0.5 * (np.pi / t)**0.5) * erf(t**0.5)

In [5]:
def potential(A, B, atom_idx):
    p, diff, K, Rp = gauss_product(A, B)
    Rc = atom_coordinates[atom_idx]
    Zc = charge_dict[atoms[atom_idx]]
    
    return (-2 * np.pi * Zc / p) * K * Fo(p * np.linalg.norm(Rp - Rc)**2)

In [6]:
def multi(A, B, C, D):
    p, diff_ab, K_ab, Rp = gauss_product(A, B)
    q, diff_cd, K_cd, Rq = gauss_product(C, D)
    multi_prefactor = 2 * np.pi**2.5 * (p * q * (p + q)**0.5)**-1
    
    return multi_prefactor * K_ab * K_cd * Fo(p * q / (p + q)*np.linalg.norm(Rp - Rq)**2)

In [ ]:
S = np.zeros((B, B))
T = np.zeros((B, B))
V = np.zeros((B, B))
multi_electron_tensor = np.zeros((B, B))

for idx_a, val_a in enumerate(atoms):
    
    Za = charge_dict[val_a]
    Ra = atom_coordinates[idx_a]
    
    for m in range(max_quantum_number[val_a]):
        